In [16]:
import pandas as pd
import google.generativeai as genai
import os
import sys
sys.path.append('..')
from config import GOOGLE_API_KEY  # Make sure this is in your config.py
import openpyxl
import requests
import io 
from io import BytesIO
import base64
import sys
from PIL import Image
from collections import defaultdict
import numpy as np
import time  # For rate limiting

# Initialize Gemini client
if not GOOGLE_API_KEY:
    raise ValueError("Please set GOOGLE_API_KEY in config.py")
genai.configure(api_key=GOOGLE_API_KEY)

# Model configuration - CHANGE THIS to use different Gemini models
MODEL_NAME = "gemini-2.0-flash-exp"  # Options: "gemini-2.0-flash-exp", "gemini-1.5-pro", "gemini-pro-vision"
model = genai.GenerativeModel(MODEL_NAME)

# ==== RATE LIMITING CONFIGURATION ====
# Gemini rate limits (as of 2025):
# - Free tier: 15 requests per minute (RPM)
# - Pay-as-you-go: 360 RPM for Flash, 60 RPM for Pro
# Adjust DELAY_BETWEEN_REQUESTS based on your tier and needs

DELAY_BETWEEN_REQUESTS = 4.0  # seconds between API calls

# Recommended delays for different tiers:
# Free tier (15 RPM): 4.0 seconds (safe margin)
# Basic tier (60 RPM): 1.0 seconds
# Standard tier (360 RPM): 0.2 seconds
# No limit needed: 0.0 seconds

# For questions with images, there are 2 API calls (real + fake image)
# So effective delay per question = DELAY_BETWEEN_REQUESTS * 2

# System prompts
SYSTEM_PROMPT = """You must reply with NO explanations, NO headers, NO extra text.
Language: ENGLISH. Keep the output strictly in the required format.

You will receive ONE medical question with:
- Text fields: QuestionID, Question, options A..E (some may be null).
- Optionally ONE image immediately after the text (when provided).

Task: choose EXACTLY ONE correct option among A, B, C, D, E for the QuestionID.
You may consider the image when present.

STRICT output format:
QuestionID, LETTER

Example:
ES3341, B

RULES:
- Output EXACTLY ONE line in the exact format above.
- Do NOT repeat the instructions.
- Do NOT include the option text, ONLY the letter."""

SYSTEM_PROMPT_WITH_REASONING = """You will receive ONE medical question with:
- Text fields: QuestionID, Question, options A..E (some may be null).
- Optionally ONE image immediately after the text (when provided).

Task: 
1. Choose EXACTLY ONE correct option among A, B, C, D, E for the QuestionID.
2. Provide your reasoning for selecting this answer.

Output format:
QuestionID, LETTER
Reasoning: [Your detailed reasoning here]

Example:
ES3341, B
Reasoning: The image shows characteristic signs of..."""

USER_LEAD = "Below is a single record. Use ONLY the relevant information."

def build_content_like_main_script(question_id: str, question_text: str, options: dict) -> list:
    """Build content in the same format as the main script"""
    content = []
    
    # Start with the lead text
    lines = [USER_LEAD, f"QuestionID: {question_id}"]
    
    # Add question text
    if question_text:
        lines.append(f"Question: {question_text}")
    
    # Add options A through E
    for label in ["A", "B", "C", "D", "E"]:
        option_text = options.get(label, "")
        lines.append(f"{label}) {option_text if option_text else ''}")
    
    # Create text content block
    content.append({
        "type": "text",
        "text": "\n".join(lines)
    })
    
    return content

def call_gemini_with_image(content, use_reasoning=False):
    """Call Gemini API and return parsed response"""
    try:
        # Build prompt parts for Gemini
        prompt_parts = []
        
        # Add system prompt
        prompt_parts.append(SYSTEM_PROMPT_WITH_REASONING if use_reasoning else SYSTEM_PROMPT)
        prompt_parts.append("\n\n")
        
        # Add content
        for item in content:
            if item["type"] == "text":
                prompt_parts.append(item["text"])
            elif item["type"] == "image":
                # Decode base64 image for Gemini
                image_data = base64.b64decode(item['source']['data'])
                img = Image.open(BytesIO(image_data))
                prompt_parts.append(img)
        
        response = model.generate_content(prompt_parts)
        
        # Add delay after API call to respect rate limits
        if DELAY_BETWEEN_REQUESTS > 0:
            time.sleep(DELAY_BETWEEN_REQUESTS)
        
        return response.text.strip()
        
    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        
        # If rate limit error, wait longer and return None
        if "429" in str(e) or "quota" in str(e).lower() or "rate limit" in str(e).lower():
            print(f"Rate limit hit! Waiting {DELAY_BETWEEN_REQUESTS * 3} seconds...")
            time.sleep(DELAY_BETWEEN_REQUESTS * 3)
        
        return None

def parse_claude_response(text):
    """Parse model's response to extract answer letter"""
    import re
    LINE_RX = re.compile(r'^\s*([^,]+)\s*,\s*([A-Ea-e])\s*$', re.UNICODE)
    
    if text:
        first_line = text.splitlines()[0] if text else ""
        m = LINE_RX.match(first_line)
        if m:
            qid_out = m.group(1).strip()
            letter = m.group(2).upper().strip()
            return letter
        else:
            return "PARSE_ERROR"
    else:
        return "NO_RESPONSE"

def process_single_repetition(df, worksheet, picture_link_col, fake_image_path, repetition_num):
    """Process all questions for one repetition"""
    print(f"\n{'#'*70}")
    print(f"### STARTING REPETITION {repetition_num} with {MODEL_NAME} ###")
    print(f"### Rate limiting: {DELAY_BETWEEN_REQUESTS}s delay between requests ###")
    print(f"{'#'*70}\n")
    
    results = []
    skipped_questions = []

    for index, row in df.iterrows():
        question_id = row['questionID']
        question_text = row['question_text']
        correct_answer = row['correct_option']
        excel_row = index + 2
        
        # Check if picture_link column exists and get its value
        picture_link = row.get('picture_link', 'N/A')
        has_image = False
        real_image = None
        
        # Try to load real image if available
        if picture_link != 'N/A' and pd.notna(picture_link):
            cell = worksheet.cell(row=excel_row, column=picture_link_col)
            if cell.hyperlink and cell.hyperlink.target:
                try:
                    url = cell.hyperlink.target
                    file_id = url.split("/d/")[1].split("/")[0]
                    download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
                    response = requests.get(download_url, timeout=30)
                    img = Image.open(BytesIO(response.content))
                    
                    # Convert RGBA to RGB
                    if img.mode in ('RGBA', 'LA'):
                        background = Image.new('RGB', img.size, (255, 255, 255))
                        background.paste(img, mask=img.split()[-1])
                        img = background
                    
                    real_image = img
                    has_image = True
                    print(f"[Rep {repetition_num}] Loaded image for {question_id}")
                except Exception as e:
                    print(f"[Rep {repetition_num}] Failed to load image for {question_id}: {e}")
        
        # Skip questions with invalid correct_option
        if pd.isna(correct_answer) or correct_answer is None:
            print(f"[Rep {repetition_num}] Skipping {question_id}: correct_option is NaN/None")
            skipped_questions.append({
                "repetition": repetition_num,
                "question_id": question_id,
                "reason": "correct_option is NaN/None"
            })
            continue

        # Convert to string and handle blanks
        correct_answer_str = str(correct_answer).strip().upper()

        if not correct_answer_str or correct_answer_str not in ['A', 'B', 'C', 'D', 'E']:
            print(f"[Rep {repetition_num}] Skipping {question_id}: Invalid correct_option '{correct_answer}'")
            skipped_questions.append({
                "repetition": repetition_num,
                "question_id": question_id,
                "reason": f"Invalid correct_option: {correct_answer}"
            })
            continue
        
        # Build options dictionary
        options = {
            "A": row['option_a'],
            "B": row['option_b'], 
            "C": row['option_c'],
            "D": row['option_d'],
            "E": row['option_e']
        }
        
        # Initialize result dictionary
        result = {
            "repetition": repetition_num,
            "question_id": question_id,
            "question": question_text,
            "correct_answer": correct_answer.upper(),
            "has_image": has_image
        }
        
        # If question has image, process with both real and fake images
        if has_image and real_image is not None:
            print(f"\n[Rep {repetition_num}] Processing question with IMAGE: {question_id}")
            
            # Build base content
            content_base = build_content_like_main_script(question_id, question_text, options)
            
            # ==== REAL IMAGE ====
            content_real = content_base.copy()
            
            # Add real image
            buffer = BytesIO()
            real_image.save(buffer, format='JPEG')
            image_data_real = base64.b64encode(buffer.getvalue()).decode('utf-8')
            
            content_real.append({
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/jpeg",
                    "data": image_data_real
                }
            })
            
            # Call Gemini with real image and reasoning
            response_real = call_gemini_with_image(content_real, use_reasoning=True)
            
            if response_real:
                claude_answer_real = parse_claude_response(response_real)
                result["claude_answer_real"] = claude_answer_real
                result["claude_response_real"] = response_real
                result["is_correct_real"] = (correct_answer.upper() == claude_answer_real)
            else:
                result["claude_answer_real"] = "API_ERROR"
                result["claude_response_real"] = "API_ERROR"
                result["is_correct_real"] = False
            
            # ==== FAKE IMAGE ====
            content_fake = content_base.copy()
            
            # Load and add fake image
            try:
                fake_img = Image.open(fake_image_path)
                if fake_img.mode in ('RGBA', 'LA'):
                    background = Image.new('RGB', fake_img.size, (255, 255, 255))
                    background.paste(fake_img, mask=fake_img.split()[-1])
                    fake_img = background
                
                buffer_fake = BytesIO()
                fake_img.save(buffer_fake, format='JPEG')
                image_data_fake = base64.b64encode(buffer_fake.getvalue()).decode('utf-8')
                
                content_fake.append({
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": image_data_fake
                    }
                })
                
                # Call Gemini with fake image and reasoning
                response_fake = call_gemini_with_image(content_fake, use_reasoning=True)
                
                if response_fake:
                    claude_answer_fake = parse_claude_response(response_fake)
                    result["claude_answer_fake"] = claude_answer_fake
                    result["claude_response_fake"] = response_fake
                    result["is_correct_fake"] = (correct_answer.upper() == claude_answer_fake)
                else:
                    result["claude_answer_fake"] = "API_ERROR"
                    result["claude_response_fake"] = "API_ERROR"
                    result["is_correct_fake"] = False
                    
            except Exception as e:
                print(f"[Rep {repetition_num}] Error loading fake image: {e}")
                result["claude_answer_fake"] = "FAKE_IMAGE_ERROR"
                result["claude_response_fake"] = f"Error: {e}"
                result["is_correct_fake"] = False
            
            print(f"[Rep {repetition_num}] {question_id}: Correct={correct_answer.upper()}, "
                  f"Real={result.get('claude_answer_real', 'N/A')}{'✓' if result.get('is_correct_real', False) else '✗'}, "
                  f"Fake={result.get('claude_answer_fake', 'N/A')}{'✓' if result.get('is_correct_fake', False) else '✗'}")
            
        else:
            # No image - process normally
            print(f"[Rep {repetition_num}] Processing question WITHOUT image: {question_id}")
            content = build_content_like_main_script(question_id, question_text, options)
            
            response_text = call_gemini_with_image(content, use_reasoning=False)
            
            if response_text:
                claude_answer = parse_claude_response(response_text)
                result["claude_answer"] = claude_answer
                result["is_correct"] = (correct_answer.upper() == claude_answer)
            else:
                result["claude_answer"] = "API_ERROR"
                result["is_correct"] = False
        
        results.append(result)
    
    return results, skipped_questions

def compute_aggregated_statistics(all_results):
    """Compute aggregated statistics with confidence intervals"""
    
    # Group results by question_id
    question_results = defaultdict(lambda: {
        'correct_answer': None,
        'has_image': False,
        'real_correct': [],
        'fake_correct': [],
        'no_image_correct': []
    })
    
    for result in all_results:
        qid = result['question_id']
        question_results[qid]['correct_answer'] = result['correct_answer']
        question_results[qid]['has_image'] = result.get('has_image', False)
        
        if result.get('has_image', False):
            question_results[qid]['real_correct'].append(result.get('is_correct_real', False))
            question_results[qid]['fake_correct'].append(result.get('is_correct_fake', False))
        else:
            question_results[qid]['no_image_correct'].append(result.get('is_correct', False))
    
    # Calculate statistics for each question
    aggregated = []
    
    for qid, data in question_results.items():
        correct = data['correct_answer']
        
        if data['has_image']:
            real_correct = data['real_correct']
            fake_correct = data['fake_correct']
            n = len(real_correct)
            
            real_correct_count = sum(real_correct)
            fake_correct_count = sum(fake_correct)
            
            real_accuracy = real_correct_count / n if n > 0 else 0
            fake_accuracy = fake_correct_count / n if n > 0 else 0
            
            # Wilson score confidence interval (95%)
            from scipy import stats
            if n > 0:
                real_ci = stats.binom.interval(0.95, n, real_accuracy) if real_accuracy > 0 else (0, 0)
                fake_ci = stats.binom.interval(0.95, n, fake_accuracy) if fake_accuracy > 0 else (0, 0)
                real_ci_lower = real_ci[0] / n
                real_ci_upper = real_ci[1] / n
                fake_ci_lower = fake_ci[0] / n
                fake_ci_upper = fake_ci[1] / n
            else:
                real_ci_lower = real_ci_upper = 0
                fake_ci_lower = fake_ci_upper = 0
            
            aggregated.append({
                'question_id': qid,
                'correct_answer': correct,
                'has_image': True,
                'num_repetitions': n,
                # Real image stats
                'real_correct_count': real_correct_count,
                'real_accuracy': real_accuracy,
                'real_ci_lower': real_ci_lower,
                'real_ci_upper': real_ci_upper,
                # Fake image stats
                'fake_correct_count': fake_correct_count,
                'fake_accuracy': fake_accuracy,
                'fake_ci_lower': fake_ci_lower,
                'fake_ci_upper': fake_ci_upper
            })
        else:
            no_image_correct = data['no_image_correct']
            n = len(no_image_correct)
            correct_count = sum(no_image_correct)
            accuracy = correct_count / n if n > 0 else 0
            
            # Wilson score confidence interval (95%)
            from scipy import stats
            if n > 0:
                ci = stats.binom.interval(0.95, n, accuracy) if accuracy > 0 else (0, 0)
                ci_lower = ci[0] / n
                ci_upper = ci[1] / n
            else:
                ci_lower = ci_upper = 0
            
            aggregated.append({
                'question_id': qid,
                'correct_answer': correct,
                'has_image': False,
                'num_repetitions': n,
                'correct_count': correct_count,
                'accuracy': accuracy,
                'ci_lower': ci_lower,
                'ci_upper': ci_upper
            })
    
    return pd.DataFrame(aggregated)

# ==== MAIN EXECUTION ====

# Configuration
NUM_REPETITIONS = 1  # Change this to desired number of repetitions
FAKE_IMAGE_PATH = "../data/Fake_Image_path/image.png"

print(f"\n{'='*70}")
print(f"Starting analysis with Google {MODEL_NAME}")
print(f"{'='*70}\n")

# Load your Excel data
df = pd.read_excel("../data/subset_with_images.xlsx", sheet_name="SSM_Q_ITA")
picture_link_col = df.columns.get_loc('picture_link') + 1
workbook = openpyxl.load_workbook("../data/subset_with_images.xlsx")
worksheet = workbook["SSM_Q_ITA"]

# Estimate time
num_questions = len(df)
questions_with_images = df['picture_link'].notna().sum()
questions_without_images = num_questions - questions_with_images

# Time calculation
# Questions with images: 2 API calls each (real + fake)
# Questions without images: 1 API call each
total_api_calls = (questions_with_images * 2 + questions_without_images) * NUM_REPETITIONS
estimated_time_seconds = total_api_calls * DELAY_BETWEEN_REQUESTS
estimated_time_minutes = estimated_time_seconds / 60
estimated_time_hours = estimated_time_minutes / 60

print(f"📊 Analysis Configuration:")
print(f"   Total questions: {num_questions}")
print(f"   Questions with images: {questions_with_images}")
print(f"   Questions without images: {questions_without_images}")
print(f"   Repetitions: {NUM_REPETITIONS}")
print(f"   Delay between requests: {DELAY_BETWEEN_REQUESTS}s")
print(f"\n⏱️  Estimated Time:")
print(f"   Total API calls: {total_api_calls}")
print(f"   Minimum time (with delays): {estimated_time_minutes:.1f} minutes ({estimated_time_hours:.1f} hours)")
print(f"\n💡 Tip: Reduce DELAY_BETWEEN_REQUESTS if you have higher rate limits")
print(f"{'='*70}\n")

# Store all results across repetitions
all_results = []
all_skipped = []

# Run multiple repetitions
for rep in range(1, NUM_REPETITIONS + 1):
    results, skipped = process_single_repetition(
        df, worksheet, picture_link_col, FAKE_IMAGE_PATH, rep
    )
    all_results.extend(results)
    all_skipped.extend(skipped)

DATETIME = time.strftime("%Y%m%d_%H%M%S")
# Save individual repetition results
all_results_df = pd.DataFrame(all_results)
all_results_df.to_csv(f"results/gemini_{MODEL_NAME}_all_repetitions_detailed_{DATETIME}.csv", index=False)
print(f"\n✓ Saved all repetitions to results/gemini_{MODEL_NAME}_all_repetitions_detailed_{DATETIME}.csv")

# Compute and save aggregated statistics
aggregated_df = compute_aggregated_statistics(all_results)
aggregated_df.to_csv(f"results/gemini_{MODEL_NAME}_aggregated_statistics_{DATETIME}.csv", index=False)
print(f"✓ Saved aggregated statistics to results/gemini_{MODEL_NAME}_aggregated_statistics_{DATETIME}.csv")

# Save skipped questions
if all_skipped:
    skipped_df = pd.DataFrame(all_skipped)
    skipped_df.to_csv(f"results/gemini_{MODEL_NAME}_skipped_questions_{DATETIME}.csv", index=False)
    print(f"✓ Saved skipped questions to results/gemini_{MODEL_NAME}_skipped_questions_{DATETIME}.csv")

# Print summary statistics
print(f"\n{'='*70}")
print(f"=== OVERALL SUMMARY for {MODEL_NAME} (across {NUM_REPETITIONS} repetitions) ===")
print(f"{'='*70}")

total_questions = len(aggregated_df)
questions_with_images = len(aggregated_df[aggregated_df['has_image'] == True])
questions_without_images = total_questions - questions_with_images

print(f"\nTotal unique questions: {total_questions}")
print(f"Questions with images: {questions_with_images}")
print(f"Questions without images: {questions_without_images}")
print(f"Total evaluations: {len(all_results)}")

if questions_with_images > 0:
    img_df = aggregated_df[aggregated_df['has_image'] == True]
    
    avg_real_accuracy = img_df['real_accuracy'].mean() * 100
    avg_fake_accuracy = img_df['fake_accuracy'].mean() * 100
    std_real_accuracy = img_df['real_accuracy'].std() * 100
    std_fake_accuracy = img_df['fake_accuracy'].std() * 100
    
    # Overall confidence interval using all data points
    total_real_correct = img_df['real_correct_count'].sum()
    total_real_trials = img_df['num_repetitions'].sum()
    total_fake_correct = img_df['fake_correct_count'].sum()
    total_fake_trials = img_df['num_repetitions'].sum()
    
    from scipy import stats
    real_overall_ci = stats.binom.interval(0.95, total_real_trials, total_real_correct/total_real_trials)
    fake_overall_ci = stats.binom.interval(0.95, total_fake_trials, total_fake_correct/total_fake_trials)
    
    print(f"\n--- REAL Images ---")
    print(f"Overall accuracy: {avg_real_accuracy:.1f}% ± {std_real_accuracy:.1f}%")
    print(f"95% CI: [{real_overall_ci[0]/total_real_trials*100:.1f}%, {real_overall_ci[1]/total_real_trials*100:.1f}%]")
    print(f"Questions always correct: {len(img_df[img_df['real_accuracy'] == 1.0])}/{questions_with_images}")
    print(f"Questions never correct: {len(img_df[img_df['real_accuracy'] == 0.0])}/{questions_with_images}")
    
    print(f"\n--- FAKE Images ---")
    print(f"Overall accuracy: {avg_fake_accuracy:.1f}% ± {std_fake_accuracy:.1f}%")
    print(f"95% CI: [{fake_overall_ci[0]/total_fake_trials*100:.1f}%, {fake_overall_ci[1]/total_fake_trials*100:.1f}%]")
    print(f"Questions always correct: {len(img_df[img_df['fake_accuracy'] == 1.0])}/{questions_with_images}")
    print(f"Questions never correct: {len(img_df[img_df['fake_accuracy'] == 0.0])}/{questions_with_images}")

if questions_without_images > 0:
    no_img_df = aggregated_df[aggregated_df['has_image'] == False]
    
    avg_accuracy = no_img_df['accuracy'].mean() * 100
    std_accuracy = no_img_df['accuracy'].std() * 100
    
    # Overall confidence interval
    total_correct = no_img_df['correct_count'].sum()
    total_trials = no_img_df['num_repetitions'].sum()
    
    from scipy import stats
    overall_ci = stats.binom.interval(0.95, total_trials, total_correct/total_trials)
    
    print(f"\n--- WITHOUT Images ---")
    print(f"Overall accuracy: {avg_accuracy:.1f}% ± {std_accuracy:.1f}%")
    print(f"95% CI: [{overall_ci[0]/total_trials*100:.1f}%, {overall_ci[1]/total_trials*100:.1f}%]")
    print(f"Questions always correct: {len(no_img_df[no_img_df['accuracy'] == 1.0])}/{questions_without_images}")
    print(f"Questions never correct: {len(no_img_df[no_img_df['accuracy'] == 0.0])}/{questions_without_images}")

print(f"\n{'='*70}")
print("Analysis complete!")
print(f"{'='*70}\n")


Starting analysis with Google gemini-2.0-flash-exp

📊 Analysis Configuration:
   Total questions: 60
   Questions with images: 60
   Questions without images: 0
   Repetitions: 1
   Delay between requests: 4.0s

⏱️  Estimated Time:
   Total API calls: 120
   Minimum time (with delays): 8.0 minutes (0.1 hours)

💡 Tip: Reduce DELAY_BETWEEN_REQUESTS if you have higher rate limits


######################################################################
### STARTING REPETITION 1 with gemini-2.0-flash-exp ###
### Rate limiting: 4.0s delay between requests ###
######################################################################

[Rep 1] Loaded image for IT0006

[Rep 1] Processing question with IMAGE: IT0006
[Rep 1] IT0006: Correct=C, Real=B✗, Fake=D✗
[Rep 1] Loaded image for IT0007

[Rep 1] Processing question with IMAGE: IT0007
[Rep 1] IT0007: Correct=C, Real=C✓, Fake=C✓
[Rep 1] Loaded image for IT0031

[Rep 1] Processing question with IMAGE: IT0031
[Rep 1] IT0031: Correct=A, Real=A✓, Fak

KeyboardInterrupt: 

In [6]:
DATETIME = time.strftime("%Y%m%d_%H%M%S")
DATETIME

'20251107_201426'

In [2]:
!pip install google.generativeai 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wandb 0.15.12 requires protobuf!=4.21.0,<5,>=3.19.0; sys_platform != "linux", but you have protobuf 5.29.5 which is incompatible.



  Obtaining dependency information for google.generativeai from https://files.pythonhosted.org/packages/6e/40/c42ff9ded9f09ec9392879a8e6538a00b2dc185e834a3392917626255419/google_generativeai-0.8.5-py3-none-any.whl.metadata
  Obtaining dependency information for google-ai-generativelanguage==0.6.15 from https://files.pythonhosted.org/packages/7c/a3/67b8a6ff5001a1d8864922f2d6488dc2a14367ceb651bc3f09a947f2f306/google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core from https://files.pythonhosted.org/packages/ed/d4/90197b416cb61cefd316964fd9e7bd8324bcbafabf40eef14a9f20b81974/google_api_core-2.28.1-py3-none-any.whl.metadata
  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/96/58/c1e716be1b055b504d80db2c8413f6c6a890a6ae218a65f178b63bc30356/google_api_python_client-2.187.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-auth>=2.15.0 from https://fil